# COGS 108 - Final Project 

In [ ]:
#!!!!!REMEMBER to delete this cell!!!!!#

client_id = 'BV-MdU-m2F5XHDfMRcQbHA'

my_key = 'CKMRLbmk9BQh_KCwAhsu9EttPeMkbQhf5cCTFMpvkLmk88fW_w4hV3CqObH2UFiJNKzcnNOQ_LYmdDLfgUyRgAmdllsJPfwQU3IprcsxE6i2bNlLAShmeGkU1YDQX3Yx'

# Overview

*3-4 sentences summarizing the project and the results*

In this project, I will: 
-find the reviews for the hotels all the parks in San Diego; then I will do a sentiment analysis on them, relate that analysis to the budget spent on these parks every year to see if spending more on parks improves overall experience of hotels

# Name & GitHub ID

- Name: Yu Pan
- GitHub Username: mikeheheda

# Research Question

*In order to help out the local economy, should the Parks Department higher their budget to create better experiences for local hotels and restaurants? In other words, is a higher budget for the Parks Department correlated to better Yelp reviews for hotels and restaurants that are close to the parks?*

## Background and Prior Work

As the coronavirus wreaks havoc on the US' economy, I am wondering if the San Diego Parks and Recreation Department should be granted a higher budget to create better park experiences, and thus help local business attract more customers. I am looking for a correlation between higher budget and better Yelp reviews on local business(specifically hotels and restaurants). While I cannot find any study that directly address this correlation, I do find many benefits that parks can bring to the local economy. 

According to WeConserve PA(1), a conservation community based in Pennsylvania, good parks attracts tourists to improve local economy. Homebuyers prefer purchasing properties near parks, which increases government tax income and local population, bringing in more business, further improing the economy. A case study demonstrates the benefits brought by the Balboa Park(2). Conducted by San Diego State University and Balboa Parj Conservancy, the *Balboa Park Benefits Study* concludes that the park alone brings an annual economic impact of 354.6 million dollars, which includes tourist lodging, food, shopping, etc. Moreover, it highlights that these non-local travelers occupy over 300 thousand hotel room nights and brings in 3.2 million dollars in sales tax. 

In a nutshell, a well-maintained parks system can generate significant economic impacts to the local economy. I am set to observe this impact on a micro scale, which is to find a correlation between the budget of the Parks Department and the reviews of local business on Yelp.

References (include links):
- 1)WeConserve PA - the Economic Benefits of Parks: https://conservationtools.org/guides/98-economic-benefits-of-parks
- 2)SDSU - Balboa Park Benefits Study: https://balboaparkconservancy.org/wp-content/uploads/2017/09/Balboa-Park-Benefits-Study-2017.pdf

# Hypothesis


Since I am looking for a relationship between the Parks spending and the quality of local businesses, I propose the alternative hypothesis: 

H<sub>1</sub>: A higher budget for the San Diego Parks Department significantly correlates with better overall Yelp reviews for businesses that are close to the parks.

and the null hypothesis:

H<sub>0</sub>: No correlation exists between San Diego Parks Departments' budget and better overall Yelp reviews for local businesses.

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [ ]:
import requests
import pandas as pd


# Data Cleaning

- Using the parks locaton dataset, find coordinates of all parks in SD
- Using above coordinates and the Yelp API, find all restaurants and hotels neraby these parks
    - Using the Yelp API, extract the reviews for all these business
- Using the Operation Actuals dataset, find the Parks department budget for the past few years

cleaning data in park_loc, producing a dataframe named df_park that contains all the park's name

In [ ]:
#Read and store parks location dataset in park_loc
park_loc_url = 'http://seshat.datasd.org/sde/parks/parks_datasd.geojson'
park_loc = requests.get(park_loc_url).json()


#cleaning data in park_loc, producing a dataframe named df_park that contains only 1)the park's name and 2) the parks' area
parks = {}
for i in range(0, len(park_loc['features']) - 1): 

    park_id = park_loc['features'][i]['properties']['objectid']
    name = park_loc['features'][i]['properties']['alias']

    parks[park_id] = name

df_park = pd.DataFrame.from_dict(data = parks, orient = 'index', columns = ['name'])


#because there is a restriction on the number of requests per day from the Yelp api, I created a new df named 'df_park_sample' that contains only the first 10 observations of 'df_park'. 
df_park_sample = df_park.iloc[:10]
print(df_park_sample)

In this following cell block, I am writing a function business_search() that will take in: 
- a park's name, 

and return 
- a list of 5 business ids that represent businesses close to the park, 

I am extracting the names and identifiers of nearby hotels and restaurants for later use

In [ ]:

def business_search(park_name): 

    ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
    PARAMS = {'term' : 'hotel', 
                'location' : park_name,
                'limit' : 5}
    HEADERS = {'Authorization' : 'bearer %s' % my_key}
    result = requests.get(url = ENDPOINT, params = PARAMS, headers = HEADERS).json()
    id_list = []

    for i in range(0, len(result['businesses'])): 
        business_id = result['businesses'][i]['id']
        id_list.append(business_id)

    return(id_list)

Apply the function above to df_park_sample to send a request to Yelp API for a list of most relevant bueiness to the location of these parks. 

In [ ]:
df_park_sample['business_ids'] = df_park_sample.apply(business_search, axis = 1)
print(df_park_sample)

Before moving on, there is a little more cleaning to do. Since I am looking at overall reviews of business near parks, I don't need to distinguish them, therefore, I will merge all the business IDs into one dataframe and eliminate any repetition

In [ ]:
#Merge all the businesses into a list
b_list = []
for i in range(1,len(df_park_sample)): 
    b_list += df_park_sample['business_ids'][i]
print('number of results returned from the search: ' + str(len(b_list)))

#Eliminate the repetition in the business ID list
bus_id_lst = []
for ind in b_list: 
    if ind not in bus_id_lst: 
        bus_id_lst.append(ind)
print('number of unique businesses in the result: ' + str(len(bus_id_lst)))
print(bus_id_lst)

Next step is to extract all the comments from yelp, and save them in a new dataframe with their comment id and comment year. 

Now I am writing a method that would take in: 
- a list of business IDs

and return:
- a dataframe object whose
    - index will be the id of the comment
    - first column will be the year of the comment
    - second column will be the content of the comment

In [ ]:
#turns the 'time_created'(str) variable to the year(int) it's created
def year_of(time): 
    return(int(time[0:4]))

def review_of(id_list): 

    review_list = pd.DataFrame(columns = ['id', 'year', 'content'])

    for id in id_list: 
        ENDPOINT = 'https://api.yelp.com/v3/businesses/' + id + '/reviews'
        HEADERS = {'Authorization' : 'bearer %s' % my_key}
        result = requests.get(url = ENDPOINT, headers = HEADERS).json()
        for num in range(0, len(result['reviews']) - 1): 
            rvw_id = result['reviews'][num]['id']
            rvw_time = result['reviews'][num]['time_created']
            rvw_text = result['reviews'][num]['text']

            x = {'id': rvw_id, 'year': year_of(rvw_time), 'content': rvw_text}
            review_list = review_list.append(x, ignore_index = True)

    return(review_list)

df_review = review_of(bus_id_lst)
print(df_review)

Next, I will wrangle the accounting data from the San Diego Parks Department: 
First step is to request it and stored it as a variable. 

In [ ]:
opr_act_url = 'http://seshat.datasd.org/budget/actuals_operating_datasd.csv'
opr_act = requests.get(url = opr_act_url).text

In [ ]:
pd.read_csv(opr_act)

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*